In [11]:
from datetime import timedelta

In [16]:
from dateutil.parser import parse

In [5]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

In [6]:
data1 = pd.read_csv("data1.csv") #초반
data1.describe()

,Voltage(V),Current(A),Resistance(Ohm),Power(W)
count,270.000000,270.000000,270.000000,270.000000
mean,3.835594,0.000439,2.811833,0.002478
std,0.028272,0.003589,18.686990,0.016470
min,3.730800,0.000000,0.000000,0.000000
25%,3.818000,0.000000,0.000000,0.000000
50%,3.844900,0.000000,0.000000,0.000000
75%,3.857450,0.000000,0.000000,0.000000
max,3.867800,0.030100,127.662000,0.114000


In [7]:
data2 = pd.read_csv("data2.csv") #중반
data2.describe()

,Voltage(V),Current(A),Resistance(Ohm),Power(W)
count,166.000000,166.000000,166.000000,166.000000
mean,3.891724,0.001087,6421.010699,0.004060
std,0.032776,0.005561,52180.347360,0.020677
min,3.722200,0.000000,0.000000,0.000000
25%,3.897700,0.000000,0.000000,0.000000
50%,3.898400,0.000000,0.000000,0.000000
75%,3.898800,0.000000,0.000000,0.000000
max,3.899000,0.030100,647645.937000,0.112000


In [8]:
data3 = pd.read_csv("data3.csv") #후반
data3.describe()

,Voltage(V),Current(A),Resistance(Ohm),Power(W)
count,130.000000,130.000000,130.000000,130.000000
mean,3.881165,0.001150,18907.774915,0.004277
std,0.037032,0.005729,98630.459572,0.021309
min,3.695700,0.000000,0.000000,0.000000
25%,3.888300,0.000000,0.000000,0.000000
50%,3.889150,0.000000,0.000000,0.000000
75%,3.889500,0.000000,0.000000,0.000000
max,3.889900,0.030000,646036.437000,0.111000


In [9]:
data = pd.read_csv("Record File__0mA(29s)_30mA(1s)(통합).csv") #통합 데이터
data.describe()

,Voltage(V),Current(A),Resistance(Ohm),Power(W)
count,4.988486e+06,4.988486e+06,4.988486e+06,4.988486e+06
mean,3.889299e+00,9.340380e-04,1.300571e+04,3.475257e-03
std,3.141821e-02,5.146647e-03,5.870463e+04,1.871865e-02
min,1.680000e-02,-1.500000e-03,0.000000e+00,0.000000e+00
25%,3.888700e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.894900e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,3.899500e+00,0.000000e+00,1.219480e+02,0.000000e+00
max,3.907000e+00,1.399000e+00,6.489843e+05,1.340000e-01


In [10]:
data = data.iloc[:, :3] #.values

In [60]:
date_values = data.values

In [12]:
data['Time'].values

array(['2021-09-24 오후 4:28:15', '2021-09-24 오후 4:28:15',
       '2021-09-24 오후 4:28:15', ..., '2021-10-15 오후 1:53:25',
       '2021-10-15 오후 1:53:25', '2021-10-15 오후 1:53:26'], dtype=object)

In [13]:
date_=[] #replace korean to english

for i in data['Time'].values:
    i = i.replace("오전", "AM")
    i = i.replace("오후", "PM")
    date_.append(i)

In [14]:
date__=[] #split blank

for i in date_:
    date__.append(i.split(" "))

In [15]:
date___=[] #switch index

for i in date__:
    tmp = i[1]
    i[1] = i[2]
    i[2] = tmp
    date___.append(i)

In [17]:
parsing_date=[]

for i in tqdm(date___):
    #print('parsing: ', i, "=", (" ").join(i))
    dt = parse((" ").join(i)) #list to string
    #print(dt)
    #print()
    parsing_date.append(dt)

계산을 해서 그거 파일의 차례대로 리스트 혹은 어레이에 저장해둔다.
그래프로 그려두고 그것으로 전압 회복 시간을 확인하는 것이 좋을 것.

1. 기준 전압 -> 시간
2. 통신 전압 -> 시간
3. 회복 전압 -> 시간
4. 전압 회복 시간 

In [54]:
str(parsing_date[2000000] - parsing_date[0])

'7 days, 1:49:23'

### 찾아야 하는 것들
1. 통신 직전의 전압 인덱스 
    - 전류가 0.020 < current (<= 0.035)인 인덱스의 전 인덱스
    ```
    np.where(data['Current(A)'] > 0.020)[0]
    # 0.020 초과인 인덱스
    ```
    
    - 전압 확인 (기준 전압)
    ```
    d_ = data['Voltage(V)']
    for idx in comm:
        print(d_[idx])
        
    ```
    - 인덱스 확인
2. 통신 전압
    - 전류가 0.020 < current인 인덱스
    - 전압 확인 (최저 전압 확인)
    - 인덱스 확인
    - 통신 전압이 기준 전압의 얼마만큼을 소비했는가
3. 통신 후 전압
    - (기준 전압 - 현재 전압) * 100
    - 전압 회복률 
    - 회복 기준이 0.5인 곳이 회복 시간이라고 하자면 시간

In [99]:
comm = np.where(data['Current(A)'] > 0.020)[0]

In [100]:
for idx in comm:
    if data['Voltage(V)'][idx] > 2.000:
        print(data['Voltage(V)'][idx])

3.7308
3.7308
3.7731
3.7731
3.7887
3.7887
3.7887
3.7967
3.7967
3.7967
3.7942
3.7942
3.8005
3.8005
3.8015
3.8015
3.8039
3.8039
3.8052
3.8052
3.8064
3.8064
3.8056
3.8056
3.8227
3.8227
3.8068
3.8068
3.8068
3.8068
3.8945
3.8071
3.8071
3.8067
3.8067
3.8045
3.8045
3.8076
3.8076
3.8052
3.8052
3.8085
3.8085
3.8052
3.8052
3.8103
3.8103
3.8042
3.8042
3.81
3.81
3.8061
3.8061
3.8099
3.8099
3.8099
3.8057
3.8057
3.8099
3.8099
3.8099
3.8062
3.8062
3.8091
3.8091
3.8064
3.8064
3.811
3.811
3.8055
3.8055
3.8055
3.8977
3.8116
3.8116
3.8056
3.8056
3.8098
3.8098
3.8068
3.8068
3.8068
3.8982
3.8129
3.8129
3.8063
3.8063
3.8115
3.8115
3.8065
3.8065
3.8114
3.8114
3.8114
3.8058
3.8058
3.8122
3.8122
3.807
3.807
3.8101
3.8101
3.8056
3.8056
3.8105
3.8105
3.8052
3.8052
3.8116
3.8116
3.8069
3.8069
3.8115
3.8115
3.8061
3.8061
3.8119
3.8119
3.806
3.806
3.8995
3.8139
3.8139
3.8054
3.8054
3.8137
3.8137
3.8054
3.8054
3.8121
3.8121
3.8058
3.8058
3.8134
3.8134
3.8056
3.8056
3.8044
3.8044
3.8069
3.8069
3.8054
3.8054
3.8059
3.

3.8034
3.7984
3.7984
3.8035
3.8035
3.7989
3.7989
3.8024
3.8024
3.7981
3.7981
3.8026
3.8026
3.8026
3.799
3.799
3.8026
3.8026
3.7983
3.7983
3.8027
3.8027
3.7982
3.7982
3.8014
3.8014
3.7991
3.7991
3.804
3.804
3.7987
3.8046
3.8046
3.7975
3.7975
3.8054
3.8054
3.7999
3.7999
3.8059
3.8059
3.7981
3.7981
3.8102
3.8102
3.7997
3.7997
3.808
3.808
3.7978
3.7978
3.7989
3.7989
3.8001
3.8001
3.8009
3.8009
3.7998
3.7998
3.7993
3.7993
3.8005
3.8005
3.8002
3.8002
3.8004
3.8004
3.7994
3.7994
3.7972
3.7972
3.8003
3.8003
3.7984
3.7984
3.8014
3.8014
3.7977
3.7977
3.8019
3.8019
3.7967
3.7967
3.8024
3.8024
3.8008
3.8008
3.8006
3.8006
3.8009
3.8009
3.7973
3.7973
3.7999
3.7999
3.8003
3.8003
3.8026
3.8026
3.7983
3.7983
3.8012
3.8012
3.7969
3.7969
3.8015
3.8015
3.7977
3.7977
3.8018
3.8018
3.7977
3.7977
3.8027
3.8027
3.7993
3.7993
3.8036
3.8036
3.798
3.798
3.8025
3.8025
3.7994
3.7994
3.8043
3.8043
3.8043
3.7985
3.7985
3.8029
3.8029
3.7991
3.7991
3.8055
3.8055
3.799
3.799
3.8042
3.8042
3.7984
3.7984
3.8047
3.8047
3.

3.7912
3.797
3.797
3.7907
3.7907
3.7959
3.7959
3.7914
3.7914
3.7959
3.7959
3.7916
3.7916
3.796
3.796
3.7919
3.7919
3.7951
3.7951
3.7919
3.7919
3.7946
3.7946
3.7937
3.7937
3.7964
3.7964
3.7916
3.7916
3.7968
3.7968
3.7924
3.7924
3.7976
3.7976
3.7918
3.7918
3.7973
3.7973
3.7915
3.7915
3.8
3.8
3.7953
3.7953
3.8001
3.8001
3.7931
3.7931
3.8007
3.8007
3.7932
3.7932
3.8014
3.8014
3.7933
3.7933
3.8074
3.8074
3.7929
3.7929
3.8042
3.8042
3.7935
3.7935
3.7933
3.7933
3.7927
3.7927
3.7945
3.7945
3.7936
3.7936
3.7935
3.7935
3.794
3.794
3.7947
3.7947
3.7946
3.7946
3.7899
3.7899
3.7899
3.796
3.796
3.7919
3.7919
3.7967
3.7967
3.793
3.793
3.7954
3.7954
3.791
3.791
3.7974
3.7974
3.7913
3.7913
3.7955
3.7955
3.7916
3.7916
3.7976
3.7976
3.7919
3.7919
3.7987
3.7987
3.7922
3.7922
3.798
3.798
3.7934
3.7934
3.8013
3.8013
3.7932
3.7932
3.8017
3.8017
3.7927
3.7927
3.7932
3.7932
3.7934
3.7934
3.7925
3.7937
3.7937
3.7931
3.7931
3.793
3.793
3.7925
3.7925
3.792
3.792
3.7923
3.7923
3.7935
3.7935
3.7948
3.7948
3.7935
3.

3.7836
3.7836
3.7906
3.7906
3.7854
3.7854
3.7904
3.7904
3.7853
3.7853
3.791
3.791
3.7846
3.7846
3.7891
3.7891
3.7859
3.7859
3.9068
3.7921
3.7921
3.7858
3.7858
3.7937
3.7937
3.7857
3.7857
3.7918
3.7918
3.7862
3.7862
3.7918
3.7918
3.7854
3.7854
3.7917
3.7917
3.7863
3.7863
3.7912
3.7912
3.7844
3.7844
3.7906
3.7906
3.7859
3.7859
3.7922
3.7922
3.7852
3.7852
3.7908
3.7908
3.7852
3.7852
3.7915
3.7915
3.7866
3.7866
3.7918
3.7918
3.7846
3.7846
3.7922
3.7922
3.7854
3.7854
3.7927
3.7927
3.7845
3.7845
3.7911
3.7911
3.7848
3.7915
3.7915
3.7861
3.7861
3.7902
3.7902
3.7848
3.7848
3.7928
3.7928
3.7841
3.7841
3.7909
3.7909
3.7859
3.7859
3.7902
3.7902
3.785
3.785
3.7892
3.7892
3.7841
3.7841
3.787
3.787
3.7843
3.7843
3.7888
3.7888
3.786
3.786
3.7907
3.7907
3.7843
3.7843
3.7897
3.7897
3.784
3.784
3.7901
3.7901
3.7829
3.7829
3.9068
3.7898
3.7898
3.7852
3.7852
3.9068
3.791
3.791
3.7856
3.7856
3.7938
3.7938
3.7856
3.7856
3.796
3.796
3.7857
3.7857
3.7851
3.7851
3.7873
3.7873
3.7868
3.7868
3.788
3.788
3.787
3.

3.7827
3.7827
3.7821
3.7821
3.7904
3.7904
3.7795
3.7795
3.7913
3.7913
3.7822
3.7822
3.7798
3.7798
3.7811
3.781
3.781
3.7799
3.7799
3.7837
3.7837
3.7811
3.7811
3.7821
3.7821
3.7822
3.7822
3.7829
3.7829
3.7826
3.7826
3.7831
3.7831
3.7824
3.7824
3.7775
3.7775
3.7839
3.7839
3.7786
3.7786
3.7823
3.7823
3.7799
3.7799
3.7846
3.7846
3.7804
3.7804
3.7833
3.7833
3.7794
3.7794
3.7841
3.7841
3.7786
3.7786
3.7853
3.7853
3.7796
3.7796
3.7828
3.7828
3.7804
3.7804
3.7839
3.7839
3.7795
3.7795
3.7836
3.7836
3.781
3.781
3.7833
3.7833
3.7818
3.7818
3.7836
3.7836
3.7796
3.7796
3.7852
3.7852
3.7807
3.7807
3.786
3.786
3.7788
3.7788
3.7849
3.7849
3.7798
3.7798
3.7844
3.7844
3.7799
3.7799
3.7847
3.7847
3.7796
3.7796
3.7852
3.7852
3.7817
3.7817
3.7887
3.7887
3.7803
3.7803
3.7851
3.7851
3.7839
3.7839
3.7849
3.7849
3.7802
3.7802
3.7846
3.7846
3.7788
3.7788
3.7871
3.7871
3.7805
3.7805
3.7849
3.7849
3.7805
3.7805
3.785
3.785
3.7798
3.7798
3.7866
3.7866
3.7775
3.7775
3.7868
3.7868
3.7827
3.7827
3.7868
3.7868
3.7781


...

3.7558
3.7558
3.7583
3.7583
3.755
3.755
3.7612
3.7612
3.7551
3.7551
3.9058
3.7601
3.7601
3.7536
3.7601
3.7601
3.7565
3.7565
3.7592
3.7592
3.7562
3.7562
3.7658
3.7658
3.7556
3.7556
3.7991
3.7991
3.7552
3.7552
3.7584
3.7584
3.7562
3.7562
3.7579
3.7579
3.7531
3.7531
3.7617
3.7617
3.7555
3.7555
3.7607
3.7607
3.7557
3.7557
3.7608
3.7608
3.7558
3.7558
3.9058
3.7635
3.755
3.755
3.7938
3.7938
3.7577
3.7577
3.7583
3.7585
3.7585
3.7601
3.7601
3.7536
3.7536
3.7575
3.7575
3.7558
3.7558
3.7609
3.7609
3.7571
3.7571
3.7633
3.7633
3.7559
3.7559
3.7585
3.758
3.758
3.7573
3.7573
3.758
3.758
3.7568
3.7568
3.7552
3.7552
3.7581
3.7581
3.7576
3.7576
3.9057
3.7609
3.7554
3.7554
3.7611
3.7611
3.7576
3.7576
3.7602
3.7602
3.7548
3.7548
3.7581
3.7581
3.7587
3.7587
3.7603
3.7603
3.7586
3.7586
3.7568
3.7568
3.7591
3.7591
3.76
3.76
3.9058
3.7569
3.7569
3.7585
3.7585
3.7545
3.7545
3.7598
3.7598
3.7568
3.7595
3.7595
3.7567
3.7567
3.7575
3.7575
3.7538
3.7538
3.7609
3.7609
3.7581
3.7612
3.7612
3.756
3.756
3.7572
3.7572

3.7549
3.7519
3.7519
3.7562
3.7562
3.7523
3.7523
3.7583
3.7583
3.7548
3.7548
3.7695
3.7695
3.7531
3.7531
3.9055
3.7544
3.7533
3.7533
3.7568
3.7568
3.751
3.751
3.7553
3.7553
3.7519
3.7519
3.7556
3.7556
3.7527
3.7527
3.7591
3.7591
3.7524
3.7524
3.7599
3.7599
3.7541
3.7541
3.7531
3.7531
3.7567
3.7567
3.7537
3.7537
3.7499
3.7499
3.7566
3.7566
3.7535
3.7535
3.757
3.757
3.7533
3.7533
3.7575
3.7575
3.7507
3.7507
3.758
3.7525
3.7525
3.7579
3.7579
3.7543
3.7543
3.7617
3.7617
3.7564
3.7564
3.7767
3.7767
3.756
3.756
3.7567
3.7567
3.7552
3.7552
3.7557
3.7557
3.7542
3.7542
3.7512
3.7512
3.7563
3.7563
3.7533
3.7558
3.7558
3.7493
3.7493
3.756
3.756
3.7517
3.7517
3.7563
3.7563
3.7491
3.7491
3.765
3.765
3.7537
3.7549
3.7549
3.7546
3.7546
3.7573
3.7573
3.7533
3.7533
3.753
3.753
3.7526
3.7526
3.7599
3.7599
3.753
3.753
3.753
3.753
3.7565
3.7565
3.7561
3.7561
3.9055
3.7519
3.7519
3.7538
3.7538
3.7502
3.7502
3.7572
3.7572
3.7506
3.7506
3.7592
3.7592
3.7543
3.7543
3.757
3.757
3.7527
3.7683
3.7683
3.757
3.757

3.7589
3.7489
3.7489
3.7531
3.7531
3.7505
3.7505
3.7515
3.7515
3.7468
3.7468
3.7548
3.7548
3.7495
3.7495
3.7575
3.7575
3.7498
3.9051
3.7641
3.7501
3.7501
3.7684
3.7684
3.7521
3.7521
3.7509
3.7509
3.7527
3.7527
3.751
3.751
3.7507
3.7507
3.7533
3.7533
3.7509
3.7509
3.7533
3.7533
3.7511
3.7511
3.754
3.754
3.7499
3.7499
3.7574
3.7574
3.7529
3.7529
3.751
3.7508
3.7508
3.7525
3.7525
3.7503
3.7503
3.7693
3.7693
3.7551
3.7551
3.7487
3.7527
3.7527
3.7488
3.7488
3.7545
3.7545
3.7508
3.7508
3.7563
3.7563
3.7514
3.7514
3.9051
3.7556
3.7556
3.7498
3.9051
3.7555
3.7555
3.7505
3.7583
3.7583
3.7515
3.7515
3.7588
3.7588
3.7501
3.7501
3.758
3.758
3.7505
3.7505
3.748
3.748
3.7543
3.7543
3.7526
3.7526
3.7478
3.7478
3.753
3.753
3.7527
3.7527
3.7555
3.7555
3.7509
3.7509
3.9051
3.7558
3.752
3.752
3.804
3.804
3.7531
3.7531
3.7518
3.7538
3.7538
3.7527
3.7527
3.7471
3.7471
3.7552
3.7552
3.7519
3.7519
3.7549
3.7549
3.7502
3.7502
3.9051
3.7593
3.7498
3.7498
3.7502
3.7502
3.7531
3.7531
3.7489
3.7489
3.7533
3.7533


3.7522
3.7522
3.7501
3.7501
3.7497
3.7497
3.7574
3.7574
3.7527
3.7527
3.7586
3.7483
3.7483
3.761
3.761
3.7492
3.7492
3.7481
3.7481
3.7536
3.7536
3.75
3.75
3.7549
3.7549
3.7485
3.7485
3.7545
3.7545
3.7497
3.7592
3.7592
3.7509
3.7494
3.7494
3.7529
3.7529
3.7519
3.7519
3.7462
3.7462
3.7522
3.7522
3.7476
3.7476
3.7519
3.7519
3.7504
3.7504
3.7544
3.7544
3.7481
3.7481
3.7514
3.7514
3.7484
3.7484
3.7619
3.7619
3.7499
3.7499
3.7509
3.7509
3.7516
3.7516
3.7486
3.7486
3.7538
3.7538
3.7477
3.7477
3.755
3.755
3.7519
3.7519
3.756
3.756
3.751
3.751
3.7577
3.7577
3.7495
3.7495
3.7508
3.7508
3.7495
3.7495
3.7504
3.7504
3.7523
3.7523
3.7482
3.7519
3.7519
3.7515
3.7515
3.7541
3.7511
3.7511
3.7529
3.7501
3.7501
3.7564
3.7504
3.7504
3.7636
3.7636
3.7526
3.7511
3.7511
3.7529
3.7529
3.748
3.748
3.7496
3.7496
3.7501
3.7501
3.7559
3.7559
3.7496
3.7496
3.7591
3.7591
3.7498
3.7498
3.7539
3.7539
3.7521
3.7521
3.7509
3.7509
3.75
3.75
3.752
3.752
3.75
3.75
3.7516
3.7516
3.7502
3.7502
3.7542
3.7542
3.7488
3.7488
3.

KeyboardInterrupt: 